In [3]:
from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

session = Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
role = get_execution_role()
print("RoleArn:", role)

region = session.boto_region_name
print("Region:", region)

RoleArn: arn:aws:iam::364723530364:role/LabRole
Region: us-east-1


In [5]:
bucket = session.default_bucket()
print(bucket)

sagemaker-us-east-1-364723530364


In [6]:
import pandas as pd

In [7]:
!ls "Car Data"

audi.csv  cclass.csv  ford.csv	  merc.csv   toyota.csv    vw.csv
bmw.csv   focus.csv   hyundi.csv  skoda.csv  vauxhall.csv


In [8]:
audi = pd.read_csv('Car Data/audi.csv')
cclass = pd.read_csv('Car Data/cclass.csv')
ford = pd.read_csv('Car Data/ford.csv')
merc = pd.read_csv('Car Data/merc.csv')
toyota = pd.read_csv('Car Data/toyota.csv')
vw = pd.read_csv('Car Data/vw.csv')
bmw = pd.read_csv('Car Data/bmw.csv')
focus = pd.read_csv('Car Data/focus.csv')
hyundi = pd.read_csv('Car Data/hyundi.csv')
skoda = pd.read_csv('Car Data/skoda.csv')
vauxhall = pd.read_csv('Car Data/vauxhall.csv')

In [9]:
datasets = [audi, cclass, ford, merc, toyota, vw, bmw, focus, hyundi, skoda, vauxhall]

In [10]:
makers = ["audi", "cclass", "ford", "merc", "toyota", "vw", "bmw", "focus", "hyundi", "skoda", "vauxhall"]

In [11]:
hyundi.rename(columns = {'tax(£)':'tax'}, inplace = True)


### Checking for Nulls in any of the datasets

In [12]:
for i,data in enumerate(datasets):
    print(data.isnull().sum().sum())
    data["maker"] = makers[i] 
#There are no null values

0
0
0
0
0
0
0
0
0
0
0


In [13]:
df_merged = pd.concat(datasets, axis=0)

In [14]:
df_merged

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,maker
0,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4,audi
1,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0,audi
2,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4,audi
3,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0,audi
4,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0,audi
...,...,...,...,...,...,...,...,...,...,...
13627,Tigra,2007,2499,Manual,48640,Petrol,165.0,46.3,1.4,vauxhall
13628,Vectra,2007,1795,Manual,75000,Petrol,240.0,39.2,1.8,vauxhall
13629,Vectra,2005,1495,Manual,74000,Petrol,235.0,38.7,1.8,vauxhall
13630,Vectra,2008,1695,Automatic,131000,Diesel,200.0,39.8,1.9,vauxhall


In [15]:
df_merged.isnull().sum()

model              0
year               0
price              0
transmission       0
mileage            0
fuelType           0
tax             9353
mpg             9353
engineSize         0
maker              0
dtype: int64

### Handeling Missing Values

In [16]:
df_merged['tax'].fillna(int(df_merged['tax'].mean()), inplace=True) #Filling them with mean
df_merged['mpg'].fillna(int(df_merged['mpg'].mean()), inplace=True) #Filling them with mean

/tmp/ipykernel_641/1043064067.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['tax'].fillna(int(df_merged['tax'].mean()), inplace=True) #Filling them with mean
/tmp/ipykernel_641/1043064067.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'd

In [17]:
df_merged.isnull().sum() #Verify there is no more missing values

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
maker           0
dtype: int64

### One Hot Encoding

In [18]:
df_merged

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,maker
0,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4,audi
1,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0,audi
2,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4,audi
3,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0,audi
4,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0,audi
...,...,...,...,...,...,...,...,...,...,...
13627,Tigra,2007,2499,Manual,48640,Petrol,165.0,46.3,1.4,vauxhall
13628,Vectra,2007,1795,Manual,75000,Petrol,240.0,39.2,1.8,vauxhall
13629,Vectra,2005,1495,Manual,74000,Petrol,235.0,38.7,1.8,vauxhall
13630,Vectra,2008,1695,Automatic,131000,Diesel,200.0,39.8,1.9,vauxhall


In [19]:
df_merged["model"].nunique() # This is to many categories to one hot encode 


195

In [20]:
High_Freq_models = df_merged["model"].value_counts()[:30]

In [21]:
High_Freq_models = High_Freq_models.index

In [22]:
df_merged.loc[~df_merged["model"].isin(High_Freq_models), "model"] = "Other" # Replace all not in the most Likely 30 with Other

In [23]:
df_encoded = pd.get_dummies(df_merged, columns=['model'])

In [24]:
df_encoded

,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,maker,model_ 1 Series,...,model_ Insignia,model_ Kuga,model_ Mokka X,model_ Octavia,model_ Polo,model_ Q3,model_ Tiguan,model_ Tucson,model_ Yaris,model_Other
0,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4,audi,False,...,False,False,False,False,False,False,False,False,False,False
1,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0,audi,False,...,False,False,False,False,False,False,False,False,False,True
2,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4,audi,False,...,False,False,False,False,False,False,False,False,False,False
3,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0,audi,False,...,False,False,False,False,False,False,False,False,False,False
4,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0,audi,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13627,2007,2499,Manual,48640,Petrol,165.0,46.3,1.4,vauxhall,False,...,False,False,False,False,False,False,False,False,False,True
13628,2007,1795,Manual,75000,Petrol,240.0,39.2,1.8,vauxhall,False,...,False,False,False,False,False,False,False,False,False,True
13629,2005,1495,Manual,74000,Petrol,235.0,38.7,1.8,vauxhall,False,...,False,False,False,False,False,False,False,False,False,True
13630,2008,1695,Automatic,131000,Diesel,200.0,39.8,1.9,vauxhall,False,...,False,False,False,False,False,False,False,False,False,True


In [25]:
df_encoded = pd.get_dummies(df_encoded, columns=['maker']) #Replace maker with one hot encoded

In [26]:
df_encoded = pd.get_dummies(df_encoded, columns=['transmission']) #Replace transmission with one hot encoded

In [27]:
df_encoded = pd.get_dummies(df_encoded, columns=['fuelType']) #Replace fueltype with one hot encoded

### Train Test and batch splitting

In [43]:
import numpy as np
rand_split = np.random.rand(len(df_encoded))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9

In [44]:
data_train = df_encoded[train_list]
data_val = df_encoded[val_list]
data_batch = df_encoded[batch_list].drop(["price"], axis=1)

In [45]:
train_file = "train_data.csv"
data_train.to_csv(train_file, index=False, header=False)

In [49]:
validation_file = "validation_data.csv"
data_val.to_csv(validation_file, index=False, header=False)

In [50]:
batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)

### Uploading to S3

In [48]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = "xgboost-final"

In [52]:
sess.upload_data(train_file, key_prefix="{}/train".format(prefix))
sess.upload_data(validation_file, key_prefix="{}/validation".format(prefix))
sess.upload_data(batch_file, key_prefix="{}/batch".format(prefix))

's3://sagemaker-us-east-1-364723530364/xgboost-final/batch/batch_data.csv'